<a href="https://colab.research.google.com/github/imemmul/Food_classification_EfficientNetB4/blob/main/food_classification_EfficientNetB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

So this colab aims to create a EfficientNetB4 model for 10 class food dataset and compare with B0

The data i get is Daniel Bourke's food data which is divided in to 3 datasets 1 percent, 10 percent and all.

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-31 10:56:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-31 10:56:34 (73.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
#importing dependencies
import tensorflow as tf
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

# Getting Data ready

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-08-31 10:56:39--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.130.128, 74.125.68.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   275MB/s    in 0.6s    

2022-08-31 10:56:40 (275 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
unzip_data("10_food_classes_10_percent.zip")

In [6]:
walk_through_dir("10_food_classes_10_percent/")

There are 2 directories and 0 images in '10_food_classes_10_percent/'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percen

In [7]:
# 75 images for training 250 images for testing

train_dir_10 = "10_food_classes_10_percent/train/"
test_dir_10 = "10_food_classes_10_percent/test/"

In [8]:
IMG_SIZE = (224, 224)
input_shape = (224, 224, 3)
train_data_10 = tf.keras.utils.image_dataset_from_directory(train_dir_10,
                                                            label_mode="categorical",
                                                            image_size=IMG_SIZE)

test_data = tf. keras.utils.image_dataset_from_directory(test_dir_10,
                                                         label_mode="categorical",
                                                         image_size=IMG_SIZE) # test_data will be always same so i am just attending it once 

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [9]:
train_data_10.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

# Building model

##### We got data ready now its time to build model i am going to build 4 different models
- model_1 is going to be our base model every tuning will be on this one.
- model_2 is going to be only fitted by augmented data.
- model_3 is going to be augmented data + tuning some layers on model_1 (which is not trainable on model_1)
- model_4 is going to be using all data + model_3 tunings

In [10]:
model = tf.keras.applications.EfficientNetB4(include_top=False)
model.trainable = False # freezing all the layers in model so that we can use them directly

inputs = tf.keras.layers.Input(shape=input_shape) # this is our input layer
x = model(inputs) # inputs passed to the model

# its time pool before outputting

x = tf.keras.layers.GlobalAveragePooling2D()(x) # tf.reduce_mean(inputs, axis=[1, 2])

outputs = tf.keras.layers.Dense(10, activation="softmax")(x) # after pooled get outputs to dense layer

71696384/71686520 [==============================] - 2s 0us/step


In [11]:
model_1 = tf.keras.Model(inputs, outputs)

model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [12]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb4 (Functional)  (None, None, None, 1792)  17673823 
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                17930     
                                                                 
Total params: 17,691,753
Trainable params: 17,930
Non-trainable params: 17,673,823
_________________________________________________________________


Before the fit i should create a checkpoint callback in order to call it while fitting

In [13]:
cp_path = "efficient_net_fit_weights/checkpoint"
cp_cb = tf.keras.callbacks.ModelCheckpoint(filepath=cp_path,
                                           save_weights_only=True,
                                           save_freq="epoch",
                                           verbose=0)

In [14]:
history_1 = model_1.fit(train_data_10,
                        validation_data=test_data,
                        steps_per_epoch=len(train_data_10),
                        validation_steps=int(0.25*(len(test_data))),
                        epochs=5) # just using 25% of test_data

Epoch 1/5
24/24 [==============================] - 34s 461ms/step - loss: 1.7847 - accuracy: 0.4933 - val_loss: 1.1426 - val_accuracy: 0.7418
Epoch 2/5
24/24 [==============================] - 8s 311ms/step - loss: 1.0182 - accuracy: 0.7480 - val_loss: 0.7952 - val_accuracy: 0.8076
Epoch 3/5
24/24 [==============================] - 8s 312ms/step - loss: 0.7618 - accuracy: 0.8187 - val_loss: 0.6736 - val_accuracy: 0.8240
Epoch 4/5
24/24 [==============================] - 8s 314ms/step - loss: 0.6300 - accuracy: 0.8413 - val_loss: 0.6242 - val_accuracy: 0.8322
Epoch 5/5
24/24 [==============================] - 8s 313ms/step - loss: 0.5529 - accuracy: 0.8587 - val_loss: 0.5838 - val_accuracy: 0.8339


Even though we used 10% of data + didn't make any differences tuning or etc. still we got 80 percent on test data.